# Extended Golay code [24,12,8]

In [200]:
import numpy as np

In [201]:
def get_G_H_matrixes():
    # returns G and H matrixes in a tuple
    
    A = np.eye(12);
    B = np.array([[1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1],
                  [0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0],
                  [0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1],
                  [1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0],
                  [1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1],
                  [1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0], 
                  [1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1],
                  [1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0],
                  [0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1],
                  [0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0],
                  [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1],
                  [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1]]);
    G = np.concatenate((A, B), axis=1);
    H = np.concatenate((B, A), axis=0);
    return (G, H);

In [202]:
(G, H) = get_G_H_matrixes();

In [203]:
def bin_sum(a, b, n):
    # n -- length of numbers
    c = a+b;
    for i in np.arange(n-1, 0, -1):
        if(c[i]==2):
            c[i]=0;
            c[i-1]+=1;
        if(c[i]==3):
            c[i]=1;
            c[i-1]+=1;
    return c;

In [204]:
def bin_multipl(A, B):
    shape_a = A.shape;
    shape_b = B.shape;
    C = np.ndarray(shape = (shape_a[0], shape_b[1]));
    
    if(shape_a[1]!=shape_b[0]):
        return 0;
    
    for i in np.arange(0, shape_a[0], 1):
        for j in np.arange(0, shape_b[1], 1):
            C[i][j] = 0;
            for k in np.arange(0, shape_a[1], 1):
                C[i][j] += A[i][k]*B[k][j];
            C[i][j] = C[i][j] % 2;
    return C;

In [205]:
def get_bin_nums_matrix():
    bin_one = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]);
    
    B = np.ndarray(shape=(4096, 12));
    B[0] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0];
    for i in np.arange(0, 4095, 1):
        next = bin_sum(B[i], bin_one, 12);
        B[i+1] = next;
    return B;

In [206]:
bin_nums = get_bin_nums_matrix();
print(bin_nums);

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  1.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 ..., 
 [ 1.  1.  1. ...,  1.  0.  1.]
 [ 1.  1.  1. ...,  1.  1.  0.]
 [ 1.  1.  1. ...,  1.  1.  1.]]


In [207]:
def get_codewords(Din, G):
    Code_words = np.ndarray(shape = (bin_nums_matrix.shape[0], 24));
    Code_words = bin_multipl(Din, G);
    return Code_words;

In [208]:
all_code_words = get_codewords(bin_nums, Mcr);
print(all_code_words);

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  1.  1.]
 [ 0.  0.  0. ...,  1.  1.  1.]
 ..., 
 [ 1.  1.  1. ...,  0.  0.  0.]
 [ 1.  1.  1. ...,  1.  0.  0.]
 [ 1.  1.  1. ...,  1.  1.  1.]]


In [209]:
# calculates a statistic on words with the same weights

import matplotlib.pyplot as plt

weight = np.zeros(4096)
hst = np.zeros(25, dtype=np.int)
print (hst)
#arr = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])
for i in range(0, 4096):
    for j in range(0, 24):
        if all_code_words[i][j] == 1:
            weight[i] += 1
print (weight)

for i in range(0, 4096):
    for j in range (0, 25):
        if weight[i] == j:
            hst[j] += 1

#plt.hist(weight)
#plt.show(weight)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[  0.   8.   8. ...,  16.  16.  24.]


In [210]:
print (hst)

[   1    0    0    0    0    0    0    0  759    0    0    0 2576    0    0
    0  759    0    0    0    0    0    0    0    1]


In [211]:
print(weight);            

[  0.   8.   8. ...,  16.  16.  24.]
